In [57]:
import datetime
from enum import Enum
import json
import os
import random
import re
import signal
import threading
import time
import hashlib
from prettytable import PrettyTable
import sys
####
from typing import Dict, List, Type, Union
import numpy as np
import torch
from torchmetrics import MeanSquaredError, MetricCollection, MeanAbsoluteError, MeanAbsolutePercentageError
from tqdm import tqdm
import wandb
from torch_timeseries.data.scaler import *
from torch_timeseries.datasets import *
from torch_timeseries.experiments.experiment import Experiment

from torch_timeseries.datasets.dataset import TimeSeriesDataset
from torch_timeseries.datasets.splitter import SequenceRandomSplitter, SequenceSplitter
from torch_timeseries.datasets.dataloader import (
    ChunkSequenceTimefeatureDataLoader,
    DDPChunkSequenceTimefeatureDataLoader,
)
from torch_timeseries.datasets.wrapper import MultiStepTimeFeatureSet
from torch_timeseries.models.Informer import Informer
from torch.nn import MSELoss, L1Loss

from torch.optim import Optimizer, Adam
from torch.utils.data import Dataset, DataLoader, RandomSampler, Subset

from torch.nn import DataParallel
import torch.nn as nn
from dataclasses import asdict,dataclass

from torch_timeseries.nn.metric import R2, Corr, TrendAcc,RMSE, compute_corr, compute_r2
from torch_timeseries.metrics.masked_mape import MaskedMAPE
from torch_timeseries.utils.early_stopping import EarlyStopping
import json
import codecs

import copy
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from class_resolver.contrib.torch import activation_resolver
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.dense.linear import Linear
from torch_geometric.nn import FAConv,HeteroConv
from torch_geometric.nn import HeteroConv, GCNConv, SAGEConv, GATConv, Linear


from torch_timeseries.layers.tcn_output8 import TCNOuputLayer

In [58]:

class EGraphSage(MessagePassing):
    """Non-minibatch version of GraphSage."""
    def __init__(self, in_channels, out_channels,
                 edge_channels=1, activation='elu', edge_mode=1,
                 normalize_emb=False, aggr='add'):
        super(EGraphSage, self).__init__(aggr=aggr)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.edge_channels = edge_channels
        self.edge_mode = edge_mode
        self.act = activation_resolver.make(activation)

        if edge_mode == 0:
            self.message_lin = nn.Linear(in_channels, out_channels)
            self.attention_lin = nn.Linear(2*in_channels+edge_channels, 1)
        elif edge_mode == 1:
            self.message_lin = nn.Linear(in_channels+edge_channels, out_channels)
        elif edge_mode == 2:
            self.message_lin = nn.Linear(2*in_channels+edge_channels, out_channels)
        elif edge_mode == 3:
            self.message_lin = nn.Sequential(
                    nn.Linear(2*in_channels+edge_channels, out_channels),
                    self.act,
                    nn.Linear(out_channels, out_channels),
                    )
        elif edge_mode == 4:
            self.message_lin = nn.Linear(in_channels, out_channels*edge_channels)
        elif edge_mode == 5:
            self.message_lin = nn.Linear(2*in_channels, out_channels*edge_channels)

        self.agg_lin = nn.Linear(in_channels+out_channels, out_channels)

        self.message_activation = self.act
        self.update_activation = self.act
        self.normalize_emb = normalize_emb

    def forward(self, x, edge_attr, edge_index):
        num_nodes = x.size(0)
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        return self.propagate(edge_index, x=x, edge_attr=edge_attr, size=(num_nodes, num_nodes))

    def message(self, x_i, x_j, edge_attr, edge_index, size):
        # x_j has shape [E, in_channels]
        # edge_index has shape [2, E]
        if self.edge_mode == 0:
            attention = self.attention_lin(torch.cat((x_i,x_j, edge_attr),dim=-1))
            m_j = attention * self.message_activation(self.message_lin(x_j))
        elif self.edge_mode == 1:
            m_j = torch.cat((x_j, edge_attr),dim=-1)
            m_j = self.message_activation(self.message_lin(m_j))
        elif self.edge_mode == 2 or self.edge_mode == 3:
            m_j = torch.cat((x_i,x_j, edge_attr),dim=-1)
            m_j = self.message_activation(self.message_lin(m_j))
        elif self.edge_mode == 4:
            E = x_j.shape[0]
            w = self.message_lin(x_j)
            w = self.message_activation(w)
            w = torch.reshape(w, (E,self.out_channels,self.edge_channels))
            m_j = torch.bmm(w, edge_attr.unsqueeze(-1)).squeeze(-1)
        elif self.edge_mode == 5:
            E = x_j.shape[0]
            w = self.message_lin(torch.cat((x_i,x_j),dim=-1))
            w = self.message_activation(w)
            w = torch.reshape(w, (E,self.out_channels,self.edge_channels))
            m_j = torch.bmm(w, edge_attr.unsqueeze(-1)).squeeze(-1)
        return m_j

    def update(self, aggr_out, x):
        # aggr_out has shape [N, out_channels]
        # x has shape [N, in_channels]
        aggr_out = self.update_activation(self.agg_lin(torch.cat((aggr_out, x),dim=-1)))
        if self.normalize_emb:
            aggr_out = F.normalize(aggr_out, p=2, dim=-1)
        return aggr_out


In [59]:
class PGraphSage(MessagePassing):
    """Non-minibatch version of GraphSage."""
    def __init__(self, in_channels, out_channels,
                 activation,
                 normalize_emb,
                 aggr):
        super(PGraphSage, self).__init__(aggr=aggr)

        self.in_channels = in_channels
        self.out_channels = out_channels

        self.message_lin = nn.Linear(in_channels, out_channels)
        self.agg_lin = nn.Linear(in_channels+out_channels, out_channels)

        self.update_activation = get_activation(activation)
        self.normalize_emb = normalize_emb

    def forward(self, x, edge_attr, edge_index):
        num_nodes = x.size(0)
        return self.propagate(edge_index, x=x, edge_attr=edge_attr, size=(num_nodes, num_nodes))

    def message(self, x_i, x_j, edge_attr, edge_index, size):
        # x_j has shape [E, in_channels]
        # edge_index has shape [2, E]
        m_j = edge_attr.view(-1,1)*x_j
        m_j = self.message_lin(m_j)
        return m_j

    def update(self, aggr_out, x):
        # aggr_out has shape [N, out_channels]
        # x has shape [N, in_channels]
        aggr_out = self.update_activation(self.agg_lin(torch.cat((aggr_out, x),dim=-1)))
        if self.normalize_emb:
            aggr_out = F.normalize(aggr_out, p=2, dim=-1)
        return aggr_out


In [60]:
import torch_geometric.nn as pyg_nn


def get_activation(activation):
    if activation == 'relu':
        return torch.nn.ReLU()
    elif activation == 'prelu':
        return torch.nn.PReLU()
    elif activation == 'tanh':
        return torch.nn.Tanh()
    elif (activation is None) or (activation == 'none'):
        return torch.nn.Identity()
    else:
        raise NotImplementedError
    
    
class GNNStack(torch.nn.Module):
    def __init__(self, 
                node_input_dim, edge_input_dim,model_types,
                node_dim=64, edge_dim=64, edge_mode=1,
                dropout=0.0, activation='relu',
                concat_states=True, node_post_mlp_hiddens=[64],
                 aggr='sum', argu_type="friend_network"
                ):
        super(GNNStack, self).__init__()
        self.dropout = dropout
        self.activation = activation
        self.concat_states = concat_states
        self.model_types = model_types
        self.argu_type = argu_type
        self.gnn_layer_num = len(model_types)
        self.normalize_embs = [True] * len(model_types)

        # convs
        self.convs = self.build_convs(node_input_dim, edge_input_dim,
                                    node_dim, edge_dim, edge_mode,
                                    model_types, self.normalize_embs, activation, aggr)
        # post node update
        if concat_states:
            self.node_post_mlp = self.build_node_post_mlp(int(node_dim*len(model_types)), int(node_dim*len(model_types)), node_post_mlp_hiddens, dropout, activation)
        else:  #default
            self.node_post_mlp = self.build_node_post_mlp(node_dim, node_dim, node_post_mlp_hiddens, dropout, activation)

        self.edge_update_mlps = self.build_edge_update_mlps(node_dim, edge_input_dim, edge_dim, self.gnn_layer_num, activation)
        if self.argu_type == 'friend_network':
            self.sage = self.build_friend_network_convs(node_dim,node_dim,activation,self.normalize_embs,aggr)
        # elif self.argu_type == 'heterogeneous_network':
        #     self.weight_han = self.build_heterogeneous_network_convs(node_dim,node_dim,activation,normalize_embs,aggr)


    def build_friend_network_convs(self, input_dim, output_dim, activation, normalize_embs, aggr):
        convs = nn.ModuleList()
        conv = PGraphSage(input_dim,output_dim,activation,normalize_embs,aggr)
        convs.append(conv)
        return convs

    # def build_heterogeneous_network_convs(self, input_dim, output_dim, activation, normalize_embs, aggr):
    #     conv = WeightedHAN(input_dim,input_dim,output_dim, n_layers=1)
    #     return conv
 
    def build_node_post_mlp(self, input_dim, output_dim, hidden_dims, dropout, activation):
        if 0 in hidden_dims:
            return get_activation('none')
        else:
            layers = []
            for hidden_dim in hidden_dims:
                layer = nn.Sequential(
                            nn.Linear(input_dim, hidden_dim),
                            get_activation(activation),
                            nn.Dropout(dropout),
                            )
                layers.append(layer)
                input_dim = hidden_dim
            layer = nn.Linear(input_dim, output_dim)
            layers.append(layer)
            return nn.Sequential(*layers)

    def build_convs(self, node_input_dim, edge_input_dim,
                     node_dim, edge_dim, edge_mode,
                     model_types, normalize_embs, activation, aggr):
        convs = nn.ModuleList()
        conv = self.build_conv_model(model_types[0],node_input_dim,node_dim,
                                    edge_input_dim, edge_mode, normalize_embs[0], activation, aggr)
        convs.append(conv)
        for l in range(1,len(model_types)):
            conv = self.build_conv_model(model_types[l],node_dim, node_dim,
                                    edge_dim, edge_mode, normalize_embs[l], activation, aggr)
            convs.append(conv)
        return convs

    def build_conv_model(self, model_type, node_in_dim, node_out_dim, edge_dim, edge_mode, normalize_emb, activation, aggr):
        if model_type == 'GCN':
            return pyg_nn.GCNConv(node_in_dim,node_out_dim)
        elif model_type == 'GraphSage':
            return pyg_nn.SAGEConv(node_in_dim,node_out_dim)
        elif model_type == 'GAT':
            return pyg_nn.GATConv(node_in_dim,node_out_dim)
        # elif model_type == 'EGCN':
        #     return EGCNConv(node_in_dim,node_out_dim,edge_dim,edge_mode)
        elif model_type == 'EGSAGE':
            return EGraphSage(node_in_dim,node_out_dim,edge_dim,activation,edge_mode,normalize_emb, aggr)
        # elif model_type == 'WeightedHAN':
        #     return EWeightedHAN(node_in_dim,node_in_dim,node_out_dim,edge_dim,n_layers=1)

    def build_edge_update_mlps(self, node_dim, edge_input_dim, edge_dim, gnn_layer_num, activation):
        edge_update_mlps = nn.ModuleList()
        edge_update_mlp = nn.Sequential(
                nn.Linear(node_dim+node_dim+edge_input_dim,edge_dim),
                get_activation(activation),
                )
        edge_update_mlps.append(edge_update_mlp)
        for l in range(1,gnn_layer_num):
            edge_update_mlp = nn.Sequential(
                nn.Linear(node_dim+node_dim+edge_dim,edge_dim),
                get_activation(activation),
                )
            edge_update_mlps.append(edge_update_mlp)
        return edge_update_mlps
    

    def update_edge_attr(self, x, edge_attr, edge_index, mlp):
        x_i = x[edge_index[0],:]
        x_j = x[edge_index[1],:]
        edge_attr = mlp(torch.cat((x_i,x_j,edge_attr),dim=-1))
        return edge_attr

    def forward(self, x, edge_attr, edge_index):
        # x : (N+M x M) node emebedding
        if self.concat_states:
            concat_x = []
        for l,(conv_name,conv) in enumerate(zip(self.model_types,self.convs)):
            if conv_name == 'EGCN' or conv_name == 'EGSAGE'  :
                x = conv(x, edge_attr, edge_index)
            else:
                x = conv(x, edge_index)
            if self.concat_states:
                concat_x.append(x)
            edge_attr = self.update_edge_attr(x, edge_attr, edge_index, self.edge_update_mlps[l])
        if self.concat_states:
            x = torch.cat(concat_x, 1)
        return x

    def F_augmentation(self, x, edge_attr, edge_index, num_obs):
        if self.argu_type == 'friend_network':
            x_sage = x[:num_obs]
            for l, conv in enumerate(self.sage):
                x_sage = conv(x_sage, edge_attr, edge_index)
            x[:num_obs] = x_sage
            return x
        # elif self.argu_type == 'heterogeneous_network':
        #     x = self.weight_han(x, edge_attr, edge_index,num_obs)
        return x


In [61]:
class MLPNet(torch.nn.Module):
    def __init__(self, 
         		input_dims, output_dim,
         		hidden_layer_sizes=(64,),
         		hidden_activation='relu',
         		output_activation=None,
                dropout=0.):
        super(MLPNet, self).__init__()

        layers = nn.ModuleList()

        input_dim = np.sum(input_dims)

        for layer_size in hidden_layer_sizes:
            hidden_dim = layer_size
            layer = nn.Sequential(
                        nn.Linear(input_dim, hidden_dim),
                        get_activation(hidden_activation),
                        nn.Dropout(dropout),
                        )
            layers.append(layer)
            input_dim = hidden_dim

        layer = nn.Sequential(
                        nn.Linear(input_dim, output_dim),
                        get_activation(output_activation),
                        )
        layers.append(layer)
        self.layers = layers

    def forward(self, inputs):
        if torch.is_tensor(inputs):
            inputs = [inputs]
        input_var = torch.cat(inputs,-1)
        for layer in self.layers:
            input_var = layer(input_var)
        return input_var






In [62]:
import torch
from torch_geometric.utils import dense_to_sparse

# 假设的 d[i]，你需要替换成你自己的数据
# 这里使用随机数据来模拟
O, N = 5, 4  # 假设的 O 和 N 的大小
d_i = torch.rand(O, N)  # 随机生成 O x N 的矩阵

# 提取边的索引和边的属性
edge_index, edge_attr = dense_to_sparse(d_i)
edge_index[1] = edge_index[1] + O

# 打印结果查看
edge_index, edge_attr


(tensor([[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4],
         [5, 6, 7, 8, 5, 6, 7, 8, 5, 6, 7, 8, 5, 6, 7, 8, 5, 6, 7, 8]]),
 tensor([0.7279, 0.6376, 0.8657, 0.9437, 0.1777, 0.7556, 0.4771, 0.8118, 0.2633,
         0.8074, 0.4561, 0.5428, 0.7921, 0.2462, 0.1073, 0.7869, 0.2728, 0.9150,
         0.7889, 0.6184]))

In [93]:
torch.autograd.set_detect_anomaly(False)

In [102]:
%load_ext autoreload
%autoreload 2

from torch_geometric.utils import dense_to_sparse

class GNNPredictor(torch.nn.Module):
    def __init__(self, feature_number, output_length, device, layers=3):
        super(GNNPredictor, self).__init__()
        
        
        self.model =  GNNStack(feature_number, 1, model_types=["EGSAGE"]*layers)
        
        self.hot_encoding_matrix = torch.eye(feature_number).detach()
        self.temporal_encoding_matrix = torch.arange(0, output_length) / output_length # 
        self.temporal_encoding_matrix = self.temporal_encoding_matrix.unsqueeze(1).repeat(1, feature_number)
        # row_indices = torch.arange(feature_number).repeat_interleave(output_length)
        # col_indices = torch.arange(feature_number, feature_number + output_length).repeat(feature_number) 
        # edge_index = torch.stack([row_indices, col_indices], dim=0).detach()
        # # bi_edge_index = torch.cat([edge_index, torch.flip(edge_index, [0])], dim=1)
        # self.sage_edge_index= torch.cat([edge_index, torch.flip(edge_index, [0])], dim=1).detach()
        # 复制 edge_attr 以匹配双向边 .to(device)
        
        self.x_input = torch.concat([self.hot_encoding_matrix, self.temporal_encoding_matrix], dim=0).to(device).detach()

        
        
        self.mlp = MLPNet(64* layers * 2, 1)
    def forward(self, d):
        # d : p - t difference
        # d: (B, O, N)
        
        B, O, N= d.size()
        x_embed = self.x_input.detach()
        
        pred_ds = []
        
        for bi in range(B):
            # 提取边的索引和边的属性
            edge_index, edge_attr = dense_to_sparse(d[bi])
            edge_index[1] = edge_index[1] + O
            edge_attr = edge_attr.unsqueeze(1)
            

            x_impute = self.model(x_embed, edge_attr, edge_index)

            pred = self.mlp([x_impute[edge_index[0]], x_impute[edge_index[1]]])
            pred = pred.reshape(O, N)
            pred_ds.append(pred)
       
        return  torch.stack(pred_ds)    
    
    
    

from torch_timeseries.models import NLinear

class NlinearGNNEnhancer(nn.Module):
    def __init__(self, seq_len, pred_len, enc_in, device, gnn_layers=3, individual=False, normalization=True):
        super(NlinearGNNEnhancer, self).__init__()

        self.nlinear = NLinear(seq_len, pred_len, enc_in, individual, normalization)
        self.gnn = GNNPredictor(enc_in, pred_len,device, layers=gnn_layers)
    
    def train_predict(self, x, y):
        p = self.nlinear(x)
        d = self.gnn(p.detach())
        # d = self.gnn(p)
        
        return p, d 
    
    
    def inference_predict(self, x):
        p = self.nlinear(x)
        d = self.gnn(p)
        
        return p + d
    

    
@dataclass
class NlinearGNNEnhanceExperiment(Experiment):
    
    
    gnn_layers : int = 3

    def _process_one_batch(self, batch_x, batch_y, batch_x_date_enc, batch_y_date_enc):
        # inputs:
        # batch_x: (B, T, N)
        # batch_y: (B, O, N)
        # ouputs:
        # - pred: (B, N)/(B, O, N)
        # - label: (B, N)/(B, O, N)
        batch_size = batch_x.size(0)
        batch_x = batch_x.to(self.device, dtype=torch.float32)
        batch_y = batch_y.to(self.device, dtype=torch.float32)
        batch_x_date_enc = batch_x_date_enc.to(self.device).float()
        batch_y_date_enc = batch_y_date_enc.to(self.device).float()
        batch_x = batch_x
        outputs = self.model.inference_predict(batch_x)  # torch.Size([batch_size, num_nodes])
        # single step prediction
        return outputs, batch_y

    def _process_one_batch_train(self, batch_x, batch_y, batch_x_date_enc, batch_y_date_enc):
        # inputs:
        # batch_x: (B, T, N)
        # batch_y: (B, O, N)
        # ouputs:
        # - pred: (B, N)/(B, O, N)
        # - label: (B, N)/(B, O, N)
        batch_size = batch_x.size(0)
        batch_x = batch_x.to(self.device, dtype=torch.float32)
        batch_y = batch_y.to(self.device, dtype=torch.float32)
        batch_x_date_enc = batch_x_date_enc.to(self.device).float()
        batch_y_date_enc = batch_y_date_enc.to(self.device).float()
        batch_x = batch_x
        p, d = self.model.train_predict(batch_x, batch_y)  # torch.Size([batch_size, num_nodes])
        # single step prediction
        return p, d , batch_y



    def _init_model(self):
        self.model = NlinearGNNEnhancer(
            seq_len=self.windows,
            pred_len=self.pred_len,
            enc_in=self.dataset.num_features,
            gnn_layers=self.gnn_layers,
            individual=True,
            normalization=True,
            device=self.device
        )
        self.model = self.model.to(self.device)
        
        self.d_loss = torch.nn.MSELoss()



    def _train(self):
        with torch.enable_grad(), tqdm(total=self.train_steps) as progress_bar:
            self.model.train()
            train_loss = []
            for i, (
                batch_x,
                batch_y,
                origin_y,
                batch_x_date_enc,
                batch_y_date_enc,
            ) in enumerate(self.train_loader):
                origin_y = origin_y.to(self.device)
                self.model_optim.zero_grad()
                pred, pred_d, true = self._process_one_batch_train(
                    batch_x, batch_y, batch_x_date_enc, batch_y_date_enc
                )
                if self.invtrans_loss:
                    pred = self.scaler.inverse_transform(pred)
                    true = origin_y
                    
                loss1 = self.d_loss(pred_d, pred.detach() - true)
                loss2 = self.loss_func(pred, true)

                loss2.backward()
                loss1.backward(retain_graph=True)

                torch.nn.utils.clip_grad_norm_(
                    self.model.parameters(), self.max_grad_norm
                )
                progress_bar.update(batch_x.size(0))
                train_loss.append(loss1.item() + loss2.item())
                progress_bar.set_postfix(
                    lr=self.model_optim.param_groups[0]["lr"],
                    epoch=self.current_epoch,
                    refresh=True,
                )

                self.model_optim.step()
            return train_loss


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [104]:
exp = NlinearGNNEnhanceExperiment(
    dataset_type="SP500",
    horizon=24,
    pred_len=1,
    data_path='/notebooks/pytorch_timeseries/data/',
    windows=40,
    
)

exp.run(456)

train steps: 1325
val steps: 333
test steps: 137
torch.get_default_dtype() torch.float32
Creating running results saving dir: './results/runs/SP500/w40h24s1/0b6d5b1f3438b07d3c634f40b4938b71'.
run : 0 in seed: 456
+---------------------------------------+------------+
|                Modules                | Parameters |
+---------------------------------------+------------+
|        nlinear.Linear.0.weight        |     40     |
|         nlinear.Linear.0.bias         |     1      |
|        nlinear.Linear.1.weight        |     40     |
|         nlinear.Linear.1.bias         |     1      |
|        nlinear.Linear.2.weight        |     40     |
|         nlinear.Linear.2.bias         |     1      |
|        nlinear.Linear.3.weight        |     40     |
|         nlinear.Linear.3.bias         |     1      |
|        nlinear.Linear.4.weight        |     40     |
|         nlinear.Linear.4.bias         |     1      |
|        nlinear.Linear.5.weight        |     40     |
|         nlinear

100%|██████████| 1325/1325 [00:03<00:00, 339.49it/s, epoch=0, lr=0.0003]


Epoch: 1 cost time: 3.9060938358306885
Traininng loss : 3.9088727150644575
Evaluating .... 


100%|██████████| 333/333 [00:01<00:00, 191.88it/s]


vali_results: {'corr': 0.2112644463777542, 'mae': 0.9795727729797363, 'mse': 1.9549514055252075, 'r2': -1.1239018440246582, 'r2_weighted': -1.1733860969543457}
Testing .... 


...

100%|██████████| 333/333 [00:01<00:00, 196.53it/s]


vali_results: {'corr': 0.2412334829568863, 'mae': 1.0363191366195679, 'mse': 1.9437885284423828, 'r2': -1.1415396928787231, 'r2_weighted': -1.1609759330749512}
Testing .... 


100%|██████████| 137/137 [00:01<00:00, 92.14it/s] 


test_results: {'corr': 0.6939830780029297, 'mae': 0.7668523788452148, 'mse': 1.206560730934143, 'r2': -1.798567295074463, 'r2_weighted': -1.1493875980377197}
Validation loss decreased (1.954951 --> 1.943789).  Saving model ...
Saving run checkpoint to './results/runs/SP500/w40h24s1/0b6d5b1f3438b07d3c634f40b4938b71'.
Run state saved ... 
torch.get_default_dtype() torch.float32


 97%|█████████▋| 1280/1325 [00:03<00:00, 465.96it/s, epoch=5, lr=0.000298]

In [40]:
predictor

GNNPredictor(
  (model): GNNStack(
    (convs): ModuleList(
      (0): EGraphSage(10, 64)
      (1-2): 2 x EGraphSage(64, 64)
    )
    (node_post_mlp): Sequential(
      (0): Sequential(
        (0): Linear(in_features=192, out_features=64, bias=True)
        (1): ReLU()
        (2): Dropout(p=0.0, inplace=False)
      )
      (1): Linear(in_features=64, out_features=192, bias=True)
    )
    (edge_update_mlps): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=129, out_features=64, bias=True)
        (1): ReLU()
      )
      (1-2): 2 x Sequential(
        (0): Linear(in_features=192, out_features=64, bias=True)
        (1): ReLU()
      )
    )
    (sage): ModuleList(
      (0): PGraphSage(64, 64)
    )
  )
  (mlp): MLPNet(
    (layers): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=384, out_features=64, bias=True)
        (1): ReLU()
        (2): Dropout(p=0.0, inplace=False)
      )
      (1): Sequential(
        (0): Linear(in_features=64, ou

In [28]:
feature_number = 10
output_length = 10
hot_encoding_matrix = torch.eye(feature_number).detach()
temporal_encoding_matrix = torch.arange(0, output_length) / output_length # 


x_input = torch.concat([hot_encoding_matrix, temporal_encoding_matrix], dim=0).detach()


RuntimeError: Tensors must have same number of dimensions: got 2 and 1

In [38]:
temporal_encoding_matrix.unsqueeze(1).repeat(1, feature_number).shape

torch.Size([10, 10])